# Multiple overlays

We have focused on data exploration on single dataframes, geared to create map visualizations for single layers. This lab goes over *overlays*, creating a single map that has multiple layers of data.

Research inquiry: Which neighborhoods in Los Angeles have the highest instances of arrests?

In [ ]:
import geopandas as gpd

In [ ]:
# get neighborhood boundaries from the LA Times
neighborhoods = gpd.read_file('http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v5.geojson')

In [ ]:
# trim the data to the bare minimum columns
neighborhoods = neighborhoods[['name','geometry']]

In [ ]:
neighborhoods.plot(figsize=(12,12),color='gainsboro', edgecolor='white')

In [ ]:
import pandas as pd
import plotly.express as px
from sodapy import Socrata

### Creating a socrata client
Next, we acquire the data using the socrata API. Use the socrata documentation to grab the code syntax for our crime data.
- https://dev.socrata.com/foundry/data.lacity.org/amvf-fr72

In [ ]:
# connect to the data portal
client = Socrata("data.lacity.org", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("amvf-fr72", 
                     limit=2000,
                     order='arst_date desc')

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

# print it with .sample, which gives you random rows
df.head()

In [ ]:
list(df)

In [ ]:
crime = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat))

In [ ]:
crime.plot(figsize=(12,12),color='red')

In [ ]:
base = neighborhoods.plot(figsize=(12,12),color='gainsboro', edgecolor='white')

ax = crime.plot(ax=base, marker='o', color='red', markersize=5)

In [ ]:
minx, miny, maxx, maxy = crime.geometry.total_bounds

In [ ]:
base = neighborhoods.plot(figsize=(12,12),color='gainsboro', edgecolor='white')
ax = crime.plot(ax=base, marker='o', color='red', markersize=5)
ax.set_xlim(minx - .1, maxx + .1) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - .1, maxy + .1)
ax

In [ ]:
type(crime)

In [ ]:
crime.set_crs(epsg=4326, inplace=True)
crime.crs

In [ ]:
join = gpd.sjoin(neighborhoods,crime,how='right')

In [ ]:
join.head()

In [ ]:
crime_by_neighborhoods = join.name.value_counts()

In [ ]:
crime_by_neighborhoods.head()

In [ ]:
crime_by_neighborhoods[:10].plot.bar()

In [ ]:
join = gpd.sjoin(neighborhoods,crime,how='right')

In [ ]:
join.shape

In [ ]:
join.plot(figsize=(12,12))